In [1]:
include("../quantum.jl")

matrixplot (generic function with 2 methods)

In [2]:
# Direct Fidelity Estimation
function estimate_fidelity_povm(ρ̂::Operator{4}, σ̂::Operator{4}, Sᵢ::Vector{Operator{4}}) 
    return 5/length(Sᵢ) * sum(ϵ -> fidelity(ρ̂, ϵ)*fidelity(ϵ, σ̂), Sᵢ)
end


estimate_fidelity_povm (generic function with 1 method)

In [31]:
function sgqt_walk_povm(ρ̂::Operator{4}, ϕᵢ::Ket{4}, measures::AbstractVector, cardinality::Int, i::Int, paramsᵦ::NamedTuple, paramsᵧ::NamedTuple, weights::Vector{Float64})
    Δᵢ = rand(Ket{4})
    Sᵢ = sample(measures, Weights(weights), cardinality, replace=false)
    βᵢ = β(i; paramsᵦ...)
    γᵢ = γ(i; paramsᵧ...)

    σ̂₊, σ̂₋ = normalize!(density(ϕᵢ + (Δᵢ*βᵢ))), normalize!(density(ϕᵢ - (Δᵢ*βᵢ)))
    F̃₊, F̃₋ = estimate_fidelity_povm(ρ̂, σ̂₊, Sᵢ), estimate_fidelity_povm(ρ̂, σ̂₋, Sᵢ)
    gᵢ = (F̃₊ - F̃₋)/2βᵢ
    return normalize(ϕᵢ + γᵢ*gᵢ*Δᵢ)
end

function sgqt_povm(ρ̂::Operator{4}, iterations::Int; measures::AbstractVector=sicpovm_matrices, cardinality::Int=8, paramsᵦ::NamedTuple=(;b=0.1, t=1/6), paramsᵧ::NamedTuple=(;a=3.0, A=0.0, s=1.0), weights::Vector{Float64}=fill(1/16, 16))
    ϕᵢ = rand(Ket{4})
    ψs = Ket{4}[]
    sizehint!(ψs, iterations)
    for i in 0:iterations-1
        push!(ψs, ϕᵢ)
        ϕᵢ₊₁ = sgqt_walk(ρ̂, ϕᵢ, measures, cardinality, i, paramsᵦ, paramsᵧ, weights)
        ϕᵢ = ϕᵢ₊₁
    end
    ψs
end

sgqt_povm(ϕ::Ket{4}, iterations::Int; measures::AbstractVector=sicpovm_matrices, cardinality::Int=8, paramsᵦ::NamedTuple=(b=0.1, t=1/6), paramsᵧ::NamedTuple=(a=3.0, A=0.0, s=1.0), weights::Vector{Float64}=fill(1/16, 16)) = sgqt(density(ϕ), iterations; measures=measures, cardinality=cardinality, paramsᵦ=paramsᵦ, paramsᵧ=paramsᵧ, weights=weights)

sgqt_povm (generic function with 2 methods)

In [34]:
fids = Base.vec(mean(hcat(map(1:100) do _ 
    ψ4 = rand(Ket{4})
    map(ψ -> fidelity(ψ4, ψ), sgqt_povm(ψ4, 100, paramsᵧ=(; a=90.0), paramsᵦ=(; b=0.35)))
end...), dims=2))

100-element Vector{Float64}:
 0.6459332658120224
 0.6172538546937264
 0.6529245473097816
 0.6880105169167966
 0.7237847270217626
 0.7372674103999034
 0.7483536293110568
 0.7680158577720769
 0.7800914649581457
 0.7887381584411562
 ⋮
 0.9231479428614372
 0.9236716929563102
 0.9241497738275348
 0.9246071294535225
 0.9250102209026483
 0.9254495594435319
 0.9257261215078502
 0.9259368933995556
 0.9265173228037901

In [35]:
lines(fids)